In [7]:
import numpy as np
import os
import tensorflow as tf

tf.reset_default_graph() # 그래프 초기화 (안하면 이미 생성된 변수와 충돌할 수 있음)
n_inputs = 3 # 입력 데이터
n_neurons = 5 # 셀의 가중치 사이즈
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
# static rnn : rnn network (4개의 셀이 연결되면서 메모리 확보)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32) # 2X4X3
Y0, Y1 = output_seqs
# 출력, 다음셀로 전달되는 값 : 마지막 state값
init = tf.global_variables_initializer()
X0_batch = np.array([[0,1,2], [3,4,5], [6,7,8], [9,0,1]])
X1_batch = np.array([[9,8,7], [0,0,0], [6,5,4], [3,2,1]])

In [9]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
print("처음 데이터 특성: ", Y0_val, "차수: ", Y0_val.shape)
print("다음 데이터 특성: ", Y1_val, "차수: ", Y1_val.shape)

처음 데이터 특성:  [[-0.1828445   0.5780428  -0.859459   -0.475783   -0.32242355]
 [-0.9245981   0.99770206 -0.9894209  -0.99872154 -0.7341757 ]
 [-0.99556625  0.9999901  -0.99925196 -0.9999978  -0.9123054 ]
 [-0.99994767  0.9958396   0.99495083 -0.99998003 -0.5827151 ]] 차수:  (4, 5)
다음 데이터 특성:  [[-0.9997636   0.99999815 -0.9936446  -1.         -0.8668509 ]
 [-0.622509   -0.86553913  0.0141169   0.8029879   0.5342773 ]
 [-0.99837846  0.99749434 -0.86618453 -0.99978346 -0.17602766]
 [-0.6327073   0.9256123   0.55491364 -0.7847594   0.43279576]] 차수:  (4, 5)


셀이 가지고 있는 가중치 사이즈는?
3*5

## Many to One

설명이 잘 되어 있당 (거의 동일한 예제)
https://github.com/golbin/TensorFlow-Tutorials/blob/master/10%20-%20RNN/01%20-%20MNIST.py

In [15]:
tf.reset_default_graph()
n_steps = 28 # 셀수
n_inputs = 28 # 셀 당 인풋 사이즈
n_neurons = 150 # 뉴론출력
n_outputs = 10 # 클래스 수
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
# 셀 28개
# state 마지막 셀의 수평으로 전달되는 값 = 마지막 cell의 output과 동일
# 28개의 셀 -> 1개의 output : many to one (감정분류같은 분류기)
# state의 차수 150X150
# output의 차수 : latent time 지연시간을 통해 계산된 셀의 모든 값을 결합 출력 150X28X150
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
# 150개 특징 중에 10개만 추출
# 입력차수와 출력차수만 지정하면 자동으로 바이어스를 생성
# 가중치 공간을 확보해서 계산
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
# 분류를 위한 미분 가능한 식으로 바뀜
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1) # 가장 큰 값을 취해서 비교
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
# 원래 이미지 모양으로 학습
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "training accuracy: ", acc_train,
             "test accuracy: ", acc_test)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 training accuracy:  0.91333336 test accuracy:  0.9333
1 training accuracy:  0.94666666 test accuracy:  0.94
2 training accuracy:  0.94666666 test accuracy:  0.9479
3 training accuracy:  0.99333334 test accuracy:  0.9622
4 training accuracy:  0.96666664 test accuracy:  0.9652
5 training accuracy:  0.96666664 test accuracy:  0.9709
6 training accuracy:  0.97333336 test accuracy:  0.9683
7 training accuracy:  0.96666664 test accuracy:  0.9721
8 training accuracy:  0.9866667 test accuracy:  0.9697
9 training accuracy:  0.9533333 test accuracy:  0.9727
10 training accuracy:  0.98 test accuracy:  0.9745
11 training accuracy:  0.99333334 test accuracy:  0.9762
12 training accuracy:  0.98 test accuracy:  0.9759
13 training accuracy:  0.98 test accuracy:  0.9755
14 training accuracy:  0.9866667 test accura

예제 소스코드
https://www.programcreek.com/python/example/102775/tensorflow.contrib.rnn.BasicRNNCell
RNN basic
https://yj-yu.github.io/rnntf/#30

In [ ]:
CNN도 텍스트마이닝을 하고, RNN도 이미지학습을 하는 등 최근에는 컨버젼스가 일어나고 있음!

# multi rnn cell

In [26]:
# BasicRNNCell
# MultiRNNCell : 수직으로 레이어 구성
# dynamic_rnn : 모델
tf.reset_default_graph()
n_steps = 28
n_inputs = 28
n_outputs = 10
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
n_neurons = 100
n_layers = 3 #3개의 멀티레이어 (3개의 셀 생성, 3개를 조합해서 MultiRNNCell 만들기)
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
# dynamic rnn : 셀로 입력되는 데이터의 개수에 맞춰 셀을 구성
# 입력 사이즈 변동 = 가중치 조절 => 나가는 특징은 일차 (neural 수가 같음)
# static rnn 동일한 사이즈 (큰 것 기준, 작은 것은 padding)
# 3층 셀이 28개 조성 : 고정 사이즈
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
# states 사이즈 3X150X100
states_concat = tf.concat(axis=1, values=states) # 150X300
logits = tf.layers.dense(states_concat, n_outputs) # 가중치 300X10 / logit 150X10
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [28]:
n_epochs = 10
batch_size = 150
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "training accuracy: ", acc_train,
             "test accuracy: ", acc_test)

0 training accuracy:  0.93333334 test accuracy:  0.9462
1 training accuracy:  0.97333336 test accuracy:  0.9604
2 training accuracy:  0.99333334 test accuracy:  0.972
3 training accuracy:  0.94666666 test accuracy:  0.9725
4 training accuracy:  0.98 test accuracy:  0.9739
5 training accuracy:  0.96666664 test accuracy:  0.9736
6 training accuracy:  0.99333334 test accuracy:  0.9788
7 training accuracy:  1.0 test accuracy:  0.9806
8 training accuracy:  0.98 test accuracy:  0.9765
9 training accuracy:  0.99333334 test accuracy:  0.9748


In [51]:
#http://blog.naver.com/PostView.nhn?blogId=magnking&logNo=221323257045&redirect=Dlog&widgetTypeCall=true&directAccess=false

import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 128X28X28
Y = tf.placeholder(tf.float32, [None, n_class]) # 128X10
W = tf.Variable(tf.random_normal([n_hidden, n_class])) #128X10
b = tf.Variable(tf.random_normal([n_class])) # 10
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # neurons : 128 (가중치 28X128)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [52]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
    print('Epoch: ', '%04d' % (epoch + 1),
         'Avg cost =', '{:.3f}'.format(total_cost/total_batch) + '최적화 완료!')

Epoch:  0001 Avg cost = 0.556최적화 완료!
Epoch:  0002 Avg cost = 0.261최적화 완료!
Epoch:  0003 Avg cost = 0.200최적화 완료!
Epoch:  0004 Avg cost = 0.171최적화 완료!
Epoch:  0005 Avg cost = 0.146최적화 완료!
Epoch:  0006 Avg cost = 0.140최적화 완료!
Epoch:  0007 Avg cost = 0.128최적화 완료!
Epoch:  0008 Avg cost = 0.121최적화 완료!
Epoch:  0009 Avg cost = 0.113최적화 완료!
Epoch:  0010 Avg cost = 0.105최적화 완료!


In [ ]:
Embedding
one-hot-encoding
tfidfcounter
CBOW
NGRAM

In [ ]:
# 문제
1. 테스트 데이터를 이용하여 테스트하는 회로를 구성하시오
2. 3개의 셀을 갖는 multilayer cell로 수정하시오

In [53]:
n_layers = 3
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_hidden, activation=tf.nn.relu) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
# 3개의 셀로 멀티레이어 생성해도 output은 하나의 셀일 경우와 동일
# state에서는 3개의 레이어마다 생성
model = tf.matmul(outputs, W) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

In [54]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print('정확도: ', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

정확도:  0.9558


In [50]:
import numpy as np
values231 = np.array([[[1],[2],[3]], [[2],[3],[4]]])
print("입력데이터의 shape", values231.shape)
import tensorflow as tf
tf.reset_default_graph()
tf_values231 = tf.constant(values231, dtype=tf.float32)
lstm_cell = tf.contrib.rnn.LSTMCell(num_units=100)
# rnn 모델은 동일 : 망과 망을 연결할 때 (번역망 : 한글, 영어)
outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell, dtype=tf.float32, inputs=tf_values231)
print("rnn이 출력하는 outputs의 의미", outputs.shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])

입력데이터의 shape (2, 3, 1)
rnn이 출력하는 outputs의 의미 (2, 3, 100)


In [55]:
print(state.c.shape) # control state
print(state.h.shape) # hidden state

(2, 100)
(2, 100)


In [58]:
tf.reset_default_graph()
values = tf.constant(np.array([[[1],[2],[3]],[[2],[3],[4]]]), dtype=tf.float32)
lstm_cell_fw = tf.contrib.rnn.LSTMCell(100)
lstm_cell_bw = tf.contrib.rnn.LSTMCell(100)
# 양방향 LSTM
(output_fw, output_bw), (output_state_fw, output_state_bw) = tf.nn.bidirectional_dynamic_rnn(
    cell_fw=lstm_cell_fw, cell_bw=lstm_cell_bw, inputs=values, dtype=tf.float32)
print(output_fw.shape)
print(output_bw.shape)

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
(2, 3, 100)
(2, 3, 100)


In [84]:
from tensorflow.contrib import rnn
import pprint
from tensorflow.python.framework import ops
ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]
# 변수 공유를 위해 지정
with tf.variable_scope('five_sequences') as scope:
    hidden_size = 2
    # 4 -> 2개 특징으로 vector화
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape) # (1,5,4) 3차원 데이터
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print(outputs.shape) # 1X5X2
    print(states.shape) # 1X2
    pp.pprint(states.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.31433782, -0.5659839 ],
        [ 0.71922415, -0.10147509],
        [-0.08392946, -0.89036334],
        [ 0.4355108 , -0.8367529 ],
        [-0.06158284, -0.59681195]]], dtype=float32)
(1, 5, 2)
(1, 2)
array([[-0.06158284, -0.59681195]], dtype=float32)


In [85]:
char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
           'q','r','s','t','u','v','w','x','y','z']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load',
           'love', 'kiss','kind']

def make_batch(seq_data):
    input_batch = []
    target_batch = []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]] # w=22, o=14, r=17
        target = num_dic[seq[-1]] # d = 3
        input_batch.append(np.eye(dic_len)[input]) # 문자를 자동으로 one-hot-encoding
        target_batch.append(target)
    return input_batch, target_batch

In [86]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합을 방지하기 위해 dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5, seed=10)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 2레이어로 구성된 멀티셀
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
# outputs : 10X3X128, states: 10X128
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b # 비선형 회귀방식
# softmax 분류
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [87]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data) # 단어데이터

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                      feed_dict={X: input_batch, Y: target_batch})
    print('Epoch: ', '%04d' % (epoch+1),
         'cost: ', '{:.6f}'.format(loss))

Epoch:  0001 cost:  3.066628
Epoch:  0002 cost:  2.243902
Epoch:  0003 cost:  1.563498
Epoch:  0004 cost:  1.191787
Epoch:  0005 cost:  0.618416
Epoch:  0006 cost:  0.699477
Epoch:  0007 cost:  0.384764
Epoch:  0008 cost:  0.441177
Epoch:  0009 cost:  0.260465
Epoch:  0010 cost:  0.303213
Epoch:  0011 cost:  0.195946
Epoch:  0012 cost:  0.131632
Epoch:  0013 cost:  0.202879
Epoch:  0014 cost:  0.446408
Epoch:  0015 cost:  0.053746
Epoch:  0016 cost:  0.080517
Epoch:  0017 cost:  0.156255
Epoch:  0018 cost:  0.135849
Epoch:  0019 cost:  0.126281
Epoch:  0020 cost:  0.055881
Epoch:  0021 cost:  0.152326
Epoch:  0022 cost:  0.237486
Epoch:  0023 cost:  0.061365
Epoch:  0024 cost:  0.015723
Epoch:  0025 cost:  0.007983
Epoch:  0026 cost:  0.017087
Epoch:  0027 cost:  0.024913
Epoch:  0028 cost:  0.049374
Epoch:  0029 cost:  0.004393
Epoch:  0030 cost:  0.019658


In [88]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy],
                                feed_dict={X: input_batch, Y: target_batch})

# 문제
## predict 한 결과를 출력하고 accuracy를 출력하시오
## 입력단어와 예측결과를 나란히 출력해보시오

In [91]:
# 츄라이...
print('예측한 결과:', predict, '\t정확도: ', accuracy_val)

for i in range(len(input_batch)): #10개 단어
    for j in range(len(input_batch[0])): #3개 알파벳
        list = []
        x = np.argmax(input_batch[i][j])
        list.append[x]

예측한 결과: [ 3  3 15  4  3 11  3  4 18  3] 	정확도:  1.0


In [113]:
# 답안
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3]+last_char)
print('입력값:', [w[:3]+' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)

# 으아닛 이리 간단했나......하하

입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


'wor'